In [5]:
#default_exp ocr_receipt

In [21]:
#export
import io
import os
from google.cloud import vision
from datetime import datetime
import argparse
from helper import perform_ocr, find_img
import re

In [7]:
#export
def parse_args():
    parser = argparse.ArgumentParser(description="OCR image processing")
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument("-r", "--random", action="store_true", help="Select a random image")
    group.add_argument("-p", "--path", type=str, help="Specify the path of the image")
    parser.add_argument("-q", "--quiet", action="store_true", help="Suppress the output")
    return parser.parse_args()

In [9]:
#export
def main():
    args = parse_args()
    client = vision.ImageAnnotatorClient()
    #  Argparse
    if args.random:
        img_path = find_img(rand_dir="../data")
    else:
        img_path = find_img(rand_dir="../data", spec_img_path=args.path)
    if not img_path:
        print("Image not found. Please check the specified path.")
        return

In [14]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../g_credential.json'
client = vision.ImageAnnotatorClient()

../data/kbank5.jpg


In [60]:
def extract_full_name(ocr_results: str):
    pattern = r'(นาย ?(?:[\u0E00-\u0E7F ]+)|นางสาว ?(?:[\u0E00-\u0E7F ]+)|น\.ส\. ?(?:[\u0E00-\u0E7F ]+)|นาง ?(?:[\u0E00-\u0E7F ]+))'
    full_name = re.findall(pattern, ocr_results)
    return full_name[0] if len(full_name) > 0 else None

def extractor(text):
    money_regex = re.compile(r'(?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d{2})?\s(?:บาท|Baht)', re.IGNORECASE)
    account_regex = re.compile(r'[Xx]+[-\s]?[\dXx]+[-\s]?[\dXx]+[-\s]?[\dXx]+', re.IGNORECASE)
    transaction_id_regex = re.compile(r'\d{12}[A-Za-z]{3}\d{5}', re.IGNORECASE)
    
    sender = extract_full_name(text)
    money = money_regex.search(text)
    account = account_regex.search(text)
    transaction_id = transaction_id_regex.search(text)

    return {
        'sender': sender,
        'money': money.group() if money else None,
        'account': account.group() if account else None,
        'transaction_id': transaction_id.group() if transaction_id else None
    }

In [61]:
img_path = find_img(rand_dir="../data")
print(f"PATH: {img_path}")
print("--------")
with io.open(img_path, "rb") as image_file:
    content = image_file.read()
texts = perform_ocr(client, content)
text = texts[0].description
print(f"TEXT:\n{text}")
print("--------")
extract_text = extractor(text)
print(f"EXTRACT TEXT:\n{extract_text}")

PATH: ../data/kbank7.jpg
--------
TEXT:
โอนเงินสําเร็จ
9 เม.ย. 66 20:51 น.
นาง สุธิดา ร
ธ.กสิกรไทย
XXX-X-x3205-x
เลขที่รายการ:
จำนวน:
นาย ปฐวี รัตนเรืองวัฒนา
ธ.กสิกรไทย
xxx-x-x0738-x
013099205128ATF02635
300.00 บาท
ค่าธรรมเนียม:
0.00 บาท
I+
บันทึกช่วยจํา: ค่าข้าว
สแกนตรวจสอบสลิป
--------
EXTRACT TEXT:
{'sender': 'นาง สุธิดา ร', 'money': '300.00 บาท', 'account': 'XXX-X-x3205-x', 'transaction_id': '013099205128ATF02635'}
